In [1]:
import random as random
import pandas as pd
import numpy as np
from skimpy import skim
from scipy import stats
import pickle

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.utils import resample
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, RFE
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor

import requests
from PIL import Image
from io import BytesIO 

import ray
import optuna
from multiprocessing import Pool, get_context
from multiprocessing.pool import ThreadPool

import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.resnet50 import preprocess_input

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, ReLU, Dropout, BatchNormalization

In [ ]:
class UberVision:

    # global path variable
    data_connection = '/uber_eats/restaurants.csv'
    author = 'Ben Stager, 2024'

    def __init__(self) -> None:
        return

    # load csv into memory
    def load_data(self, cols=''):  
        self.cols = cols 
        if cols == '':
            return pd.read_csv(UberVision.data_connection)

    def load_image(self, url_arr):
        self.url_arr = url_arr
        
        if url_arr is list():
            self.url_arr = url_arr
            images = []
            print(url_arr)

            for url in url_arr:
                response = requests.get(url)
                img = Image.open(BytesIO(response.content))
                images.append(img)

            return images
        
        self.url_arr = url_arr
        images = []
        print(url_arr)
    
    def extract_feature(self, img):
        self.img_arr = img

        resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        features_img = resnet.predict(img_array)[0]

        return features_img
    
    
    def prepare_extract_features(self, img_arr):

        self.img_arr = img_arr
        features = []

        resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

        for img in img_arr:
            img_array = img_to_array(img)
            img_array = np.expand_dims(img_array, axis=0)
            img_array = preprocess_input(img_array)

            features_img = resnet.predict(img_array)

            features.extend(features_img[0])

        return features
    
    def UberBaseline():
        pass
        
    def UberXGBoost():
        pass

    def UberRandomForest():
        pass

    def UberBaseline():
        pass

In [ ]:
def load_image(url_arr):
    images = []
    print(url_arr)

    for url in url_arr:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        images.append(img)

    return images

In [ ]:
def prepare_extract_features(img_arr):
    features = []

    resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    for img in img_arr:
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        features_img = resnet.predict(img_array)

        features.extend(features_img[0])

    return features

In [ ]:
def run_PCA(X_train, X_test, use_all=True, num_to_use=3, num_components=3):
    
    if use_all:
        pca = PCA()
        X_train_PCA = pca.fit_transform(X_train)[:,:num_to_use]
        X_test_PCA = pca.transform(X_test)[:,:num_to_use]

        return X_train_PCA, X_test_PCA
    
    pca = PCA(num_components)
    return pca.fit_transform(X_train), pca.transform(X_test)

In [46]:
def convert_price(x):
    prices_dict = {'$':0, '$$':1}
    return prices_dict[x]

In [44]:
img_cols = []
img_cols.append('loc_no_address')
img_cols.append('loc_name')
[img_cols.append(f'img{i}') for i in range(1,6)]
img_cols.append('review_rating')
img_cols.append('price_bucket')

df = pd.read_csv('Ubereat_US_Merchant.csv')
df['loc_name'] = df['loc_name'].apply(lambda x: x.lstrip())
df['loc_name'] = df['loc_name'].apply(lambda x: x.rstrip())
df = df.dropna(subset=img_cols[2:7])
df['loc_no_address'] = df['loc_name'].apply(lambda x: x.split('(')[0])

df = df[img_cols]

df = df.groupby('loc_no_address').agg({'img1':'first','img2':'first','img3':'first','img4':'first','img5':'first','review_rating':'mean', 'price_bucket':'first'}).reset_index()

df_labeled = df[df['review_rating'].isna() == False]
df_unlabeled = df[df['review_rating'].isna() == True]

features = pd.read_csv('design_matrix.csv')

df_training = pd.concat([df_labeled.reset_index().drop('index',axis=1), features.reset_index().drop(['index', 'Unnamed: 0'],axis=1)],axis=1)

one = df_training.drop('review_rating',axis=1)
two = df_training['review_rating']

df_training = pd.concat([one,two],axis=1)
df_training = df_training[df_training['price_bucket'].isna() == False]
df_training = df_training[df_training['price_bucket'] != '$$$']
df_training_X = df_training.drop(['loc_no_address', 'review_rating', 'img1', 'img2', 'img3', 'img4', 'img5', 'price_bucket'],axis=1)
df_training_y = df_training['review_rating']

In [ ]:
data = pd.read_csv('Ubereat_US_Merchant.csv')

In [47]:
df_training_X = df_training.drop(['loc_no_address', 'review_rating', 'img1', 'img2', 'img3', 'img4', 'img5', 'price_bucket'],axis=1)
df_training_y = df_training['price_bucket'].apply(convert_price)

df_majority = df_training[df_training['price_bucket'] == '$']
df_minority = df_training[df_training['price_bucket'] == '$$']

df_majority_downsampled = resample(df_majority,
                                    replace=False, 
                                    n_samples=len(df_minority), 
                                    random_state=42)

downsampled_df = pd.concat([df_majority_downsampled, df_minority])

df_training_X = downsampled_df.drop(['loc_no_address', 'review_rating', 'img1', 'img2', 'img3', 'img4', 'img5', 'price_bucket'],axis=1)
df_training_y = downsampled_df['price_bucket'].apply(convert_price)

In [305]:
X_train, X_test, y_train, y_test = train_test_split(df_training_X, df_training_y, test_size=.25)

In [295]:
X_train_PCA, X_test_PCA = run_PCA(X_train, X_test, use_all=False, num_components=7)

In [55]:
for j in range(5):
   
    pca = PCA(n_components=12)
    pca.fit(df_training_X[[str(i) for i in range(2048*j, 2048 + 2048*j)]])
    with open(f'pca_model{j}.pkl', 'wb') as file:
        pickle.dump(pca, file)

In [306]:
y_preds_baseline = np.tile(1, y_test.shape[0])
baseline_accuracy = y_test[y_test == 1].shape[0]/y_test.shape[0]

print(baseline_accuracy)

0.48


In [301]:
model = SVC(kernel='poly')
model.fit(X_train_PCA, y_train)
y_preds_SVC = model.predict(X_test_PCA)
error_SVC = accuracy_score(y_test, y_preds_SVC)

print(error_SVC)

0.48


In [300]:
model = LogisticRegression()
model.fit(X_train_PCA, y_train)
y_preds_LR = model.predict(X_test_PCA)
error_LR = accuracy_score(y_test, y_preds_LR)

print(error_LR)

0.48


In [334]:
y_preds_final = []

for j in range(5):
    #f_training_X_img = df_training_X[[str(j) for j in range(2048*j, 2048 + 2048*j)]]
    #df_training_y_img = downsampled_df['price_bucket'].apply(convert_price)
    #X_train, X_test, y_train, y_test = train_test_split(df_training_X_img, df_training_y_img, test_size=.25, random_state=22)
    X_train_PCA_meta, X_test_PCA_meta = run_PCA(X_train[[str(i) for i in range(2048*j, 2048 + 2048*j)]], X_test[[str(i) for i in range(2048*j, 2048 + 2048*j)]], use_all=False, num_components=12)

    def objective(trial):
        params = {
            'n_estimators': trial.suggest_int('n_estimators', 10, 300),          
            'max_depth': trial.suggest_int('max_depth', 10, 250),            
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 15, step=1),  
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 2, 8, step=1),      
            'bootstrap': trial.suggest_categorical('bootstrap', [True, False])    
        }

        model = RandomForestClassifier(**params)
        model.fit(X_train_PCA_meta, y_train)
        y_preds_RF = model.predict(X_test_PCA_meta)

        error = accuracy_score(y_test, y_preds_RF)

        return error

    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=100)

    model = RandomForestClassifier(**study.best_params)

    model.fit(X_train_PCA_meta, y_train)

    with open(f'random_forest_model_{j}.pkl', 'wb') as file:
       pickle.dump(model, file)
    
    y_preds_RF = model.predict(X_test_PCA_meta)
    y_preds_final.append(y_preds_RF)

meta_matrix = pd.DataFrame(y_preds_final)
y_preds_META_MODEL = [meta_matrix[i].mode().squeeze() for i in range(25)]
error_RF_meta = accuracy_score(y_test, y_preds_META_MODEL)

[I 2024-10-20 20:57:26,991] A new study created in memory with name: no-name-61f0e862-9934-4e5e-bfd1-e53009d9a0da
[I 2024-10-20 20:57:27,177] Trial 0 finished with value: 0.6 and parameters: {'n_estimators': 289, 'max_depth': 16, 'min_samples_split': 11, 'min_samples_leaf': 3, 'bootstrap': False}. Best is trial 0 with value: 0.6.
[I 2024-10-20 20:57:27,279] Trial 1 finished with value: 0.68 and parameters: {'n_estimators': 164, 'max_depth': 174, 'min_samples_split': 9, 'min_samples_leaf': 5, 'bootstrap': False}. Best is trial 1 with value: 0.68.
[I 2024-10-20 20:57:27,297] Trial 2 finished with value: 0.6 and parameters: {'n_estimators': 37, 'max_depth': 149, 'min_samples_split': 7, 'min_samples_leaf': 5, 'bootstrap': True}. Best is trial 1 with value: 0.68.
[I 2024-10-20 20:57:27,336] Trial 3 finished with value: 0.68 and parameters: {'n_estimators': 84, 'max_depth': 204, 'min_samples_split': 10, 'min_samples_leaf': 3, 'bootstrap': True}. Best is trial 1 with value: 0.68.
[I 2024-10-2

In [335]:
error_RF_meta

0.64

In [ ]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)

set_seed(22)

def create_MLP(num_layers=2, dropout_rate=.5, units=16):
    model = Sequential()
    model.add(Dense(units=32, input_dim=X_train_PCA.shape[1]))
    for i in range(num_layers):
        model.add(Dense(units, activation='tanh'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    return model

def objective_nn(trial):
    num_layers = trial.suggest_int('num_layers', 1, 3, step=2)
    dropout_rate = trial.suggest_uniform('dropout_rate', .2, .7)
    units = trial.suggest_int('units', 1, 16, step=4)
    model = create_MLP(num_layers, dropout_rate, units)
    model.fit(X_train_PCA, y_train, epochs=200)
    y_preds_MLP = model.predict(X_test_PCA)
    y_preds_MLP = (y_preds_MLP > 0.5).astype(int)
    
    loss = accuracy_score(y_test, y_preds_MLP)
    
    return loss

study = optuna.create_study(direction='maximize')
study.optimize(objective_nn, n_trials=20)

set_seed(22)

best_parms = study.best_params
model = create_MLP(**best_parms)

model.fit(X_train_PCA, y_train, epochs=200)
y_preds_MLP = model.predict(X_test_PCA)
y_preds_MLP = (y_preds_MLP > 0.5).astype(int)
error_NN_hyperparam = accuracy_score(y_test, y_preds_MLP)

print(error_NN_hyperparam)

In [239]:
model = Sequential()

model.add(Dense(8, input_dim = X_train_PCA.shape[1], activation='tanh'))
model.add(Dropout(0.5)) 
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy')

model.fit(X_train_PCA, y_train, epochs=200)

y_preds_MLP = model.predict(X_test_PCA)
y_preds_MLP = (y_preds_MLP > 0.5).astype(int)
error_NN = accuracy_score(y_test, y_preds_MLP)

Epoch 1/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7392
Epoch 2/200
3/3 [==============================] - 0s 926us/step - loss: 0.7416
Epoch 3/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7548
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7344
Epoch 5/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7953
Epoch 6/200
3/3 [==============================] - 0s 988us/step - loss: 0.7533
Epoch 7/200
3/3 [==============================] - 0s 844us/step - loss: 0.8362
Epoch 8/200
3/3 [==============================] - 0s 851us/step - loss: 0.7679
Epoch 9/200
3/3 [==============================] - 0s 1ms/step - loss: 0.8335
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 0.8551
Epoch 11/200
3/3 [==============================] - 0s 3ms/step - loss: 0.7998
Epoch 12/200
3/3 [==============================] - 0s 5ms/step - loss: 0.7054
Epoch 13/200
3/3 [==============================] - 0

In [240]:
error_NN

0.48

In [321]:
y_preds_final = []
for j in range(5):
    #f_training_X_img = df_training_X[[str(j) for j in range(2048*j, 2048 + 2048*j)]]
    #df_training_y_img = downsampled_df['price_bucket'].apply(convert_price)
    #X_train, X_test, y_train, y_test = train_test_split(df_training_X_img, df_training_y_img, test_size=.25, random_state=22)
    X_train_PCA_meta, X_test_PCA_meta = run_PCA(X_train[[str(j) for j in range(2048*j, 2048 + 2048*j)]], X_test[[str(j) for j in range(2048*j, 2048 + 2048*j)]], use_all=False, num_components=10)
    
    model = Sequential()

    model.add(Dense(32, input_dim = X_train_PCA_meta.shape[1], activation='tanh'))
    model.add(Dropout(0.5)) 
    model.add(Dense(16, input_dim = X_train_PCA_meta.shape[1], activation='tanh'))
    model.add(Dropout(0.5)) 
    model.add(BatchNormalization())
    model.add(Dense(8, input_dim = X_train_PCA_meta.shape[1], activation='tanh'))
    model.add(Dropout(0.5)) 
    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy')

    model.fit(X_train_PCA_meta, y_train, epochs=200)

    y_preds_MLP = model.predict(X_test_PCA_meta).squeeze()
    y_preds_MLP = (y_preds_MLP > 0.5).astype(int)
    y_preds_final.append(y_preds_MLP)

meta_matrix = pd.DataFrame(y_preds_final)
y_preds_META_MODEL = [meta_matrix[i].mode().squeeze() for i in range(25)]
error_NN_meta = accuracy_score(y_test, y_preds_META_MODEL)

Epoch 1/200
3/3 [==============================] - 0s 2ms/step - loss: 0.8073
Epoch 2/200
3/3 [==============================] - 0s 1ms/step - loss: 0.8544
Epoch 3/200
3/3 [==============================] - 0s 985us/step - loss: 0.6771
Epoch 4/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7561
Epoch 5/200
3/3 [==============================] - 0s 920us/step - loss: 0.8087
Epoch 6/200
3/3 [==============================] - 0s 1ms/step - loss: 0.8370
Epoch 7/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7823
Epoch 8/200
3/3 [==============================] - 0s 2ms/step - loss: 0.7346
Epoch 9/200
3/3 [==============================] - 0s 987us/step - loss: 0.6908
Epoch 10/200
3/3 [==============================] - 0s 826us/step - loss: 0.7263
Epoch 11/200
3/3 [==============================] - 0s 989us/step - loss: 0.8434
Epoch 12/200
3/3 [==============================] - 0s 1ms/step - loss: 0.7841
Epoch 13/200
3/3 [==============================] -

In [322]:
error_NN_meta

0.52

In [2]:
model_path = '../app/random_forest_model_0.pkl'

with open(model_path, 'rb') as file:
    test = pickle.load(file)

In [8]:
stats.mode([1,2,2])[0][0]

/var/folders/k_/y5wzmsx1483f9z9t9tjtdj1m0000gn/T/ipykernel_15381/3265576334.py:1: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  stats.mode([1,2,2])[0][0]


2

In [ ]:
def run_model(features):
    predictions = []
    for j in range(5):
        model_path = f'../app/random_forest_model_{j}.pkl'

        with open(model_path, 'rb') as file:
            model = pickle.load(file)

        prediction = model.predict(features[:, 2048*j: 2048 + 2048*j])
        predictions.append(prediction)

    return stats.mode(predictions)[0][0]


In [ ]:
def load_image(url_arr):
    images = []
    print(url_arr)

    for url in url_arr:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        images.append(img)

    return images

In [14]:
def load_image(url_arr):
    images = []

    for url in url_arr:
        img = Image.open(url)
        images.append(img)

    return images

In [10]:
url_arr = ['test2.jpeg' for _ in range(5)]

In [15]:
test = load_image(url_arr=url_arr)

['test2.jpeg', 'test2.jpeg', 'test2.jpeg', 'test2.jpeg', 'test2.jpeg']


In [17]:
def prepare_extract_features(img_arr):
    features = []

    resnet = ResNet50(weights='imagenet', include_top=False, pooling='avg')

    for img in img_arr:
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        features_img = resnet.predict(img_array)

        features.extend(features_img[0])

    return features

In [18]:
feat = prepare_extract_features(img_arr=test)

1/1 [==============================] - 0s 277ms/step


In [25]:
j = 4

In [26]:
feat[:2048 + 2048*j]

[0.31222063,
 0.17221065,
 1.3446822,
 0.010240825,
 0.43195227,
 0.031182546,
 0.0155016845,
 0.029019369,
 0.42756066,
 0.014874036,
 0.32372195,
 0.18074471,
 0.46788847,
 0.7044107,
 0.14528532,
 0.028149676,
 0.01974666,
 0.0028312386,
 0.47781754,
 0.1713444,
 0.11797799,
 0.06346274,
 0.2503104,
 0.40074503,
 0.02807137,
 1.3308756,
 0.27151337,
 1.1274112,
 0.008978986,
 4.0886617,
 0.017905775,
 0.30960453,
 0.48425302,
 0.054721788,
 0.13850228,
 0.047077816,
 0.18923755,
 0.0889336,
 0.23684281,
 0.063227616,
 0.03030495,
 0.413069,
 0.16253786,
 0.12764983,
 0.20045128,
 0.040668733,
 0.16858797,
 0.47242272,
 0.004032733,
 0.3395215,
 0.7512693,
 0.2626947,
 0.029037146,
 0.06817128,
 0.63301104,
 0.118644856,
 0.06369893,
 0.5486805,
 0.10635554,
 0.26945332,
 0.37883624,
 0.10667766,
 0.882942,
 0.03948549,
 0.032641944,
 0.9799549,
 0.0206137,
 0.045486562,
 0.069370136,
 0.3966892,
 0.15802743,
 0.15688881,
 1.0275323,
 0.58452004,
 0.09528443,
 0.17645091,
 0.32551718

In [42]:
def run_PCA_prod(features, use_all=False, num_to_use=3, num_components=12):
    
    if use_all:
        pca = PCA()
        X_train_PCA = pca.fit_transform(features)[:,:num_to_use]
        X_test_PCA = pca.transform(features)[:,:num_to_use]

        return X_train_PCA, X_test_PCA
    
    pca = PCA(num_components)
    return pca.transform(features)

In [68]:
def run_model(features):
    predictions = []
    
    for j in range(5):
        model_path = f'../app/random_forest_model_{j}.pkl'
        file_path = f'../model/pca_model{j}.pkl'
        
        with open(file_path, 'rb') as file:
            pca = pickle.load(file)
        with open(model_path, 'rb') as file:
            model = pickle.load(file)
            
        feature_vector = pca.transform(np.array(features[2048*j:2048 + 2048*j]).reshape(1,-1))
        prediction = model.predict(np.array(feature_vector).reshape(1,-1))
        predictions.append(prediction)
        
    return stats.mode(predictions)[0][0].squeeze()

In [69]:
pred = run_model(feat)

1


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but PCA was fitted with f

In [65]:
pred

array(1)